# 4.9 Intro to Data Visualization Exercise

# Step 1, 2, and 3 Prepare Environment

In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

In [2]:
# Create path

path = r"C:\Users\leobs\OneDrive\Data Program\Achievement 4_Instacart\Instacart Basket Analysis"

In [3]:
# Import prior dataframe containing orders and products merged, from exercise 4.8

ords_prods_merge_final = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_final.pkl'))

#Note, this file includes the merged data sets from orders and products, aggregated columns, and outliers removed

In [4]:
# Check import

ords_prods_merge_final.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spending_flag,median_last_order,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [5]:
# Check shape

ords_prods_merge_final.shape

(32404859, 24)

In [6]:
# Import new customer dataframe

df_cust = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'customers.csv'))

In [7]:
# Check import

df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [8]:
# Check shape

df_cust.shape

(206209, 10)

# EDA with new customer dataframe

In [9]:
# Check stats

df_cust.describe()

,user_id,Age,n_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


Observations The data appears complete because the counts for all numerical categories match The mean age shows a tendency for middle aged customers as the based There is a wide spread for income There is a small range for number of dependents All minimums appear accurate All maximum values appear accurate, though work checking the top tier incomes for outliers due to spread

In [10]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   user_id       206209 non-null  int64 
 1   First Name    194950 non-null  object
 2   Surnam        206209 non-null  object
 3   Gender        206209 non-null  object
 4   STATE         206209 non-null  object
 5   Age           206209 non-null  int64 
 6   date_joined   206209 non-null  object
 7   n_dependants  206209 non-null  int64 
 8   fam_status    206209 non-null  object
 9   income        206209 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 15.7+ MB


Observations first_name column only has 194950 non-null entries compared to 206209, need to investigate further during quality checks

# Wrangling customer data

## Drop columns

No need to remove columns at this time because all of the information could be relevant to finding customer trends for Instacart

## Rename columns

Need to match formate on columns to lower case and no spaces. # Match formate on columns to lower case and no spaces
family_status column only includes marital status, not partner/kids/etc. This column is not complete because it does not include all partnerships outside of marriage, assumes you are living with your partner if you are married, and excludes living alone and/or with platonic roommates. It also assumes you live with your partner if you are married. The 'Living with parents and siblings' value is not congruent with the rest of the values and can be discussed with the client. 
Without more information, it is unlikely that there were gender options for gender diverse people to self-identify. There are also only two binary genders listed. We don't have accurate "gender" information at this time so the rename to sex is more accurate. 
Renaming n_dependants for clarity to number_of_dependants

In [11]:
df_cust.head()

,user_id,First Name,Surnam,Gender,STATE,Age,date_joined,n_dependants,fam_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [12]:
df_cust.rename(columns = {'First Name' : 'first_name'}, inplace = True)

In [13]:
df_cust.rename(columns = {'Surnam' : 'last_name'}, inplace = True)

In [14]:
df_cust.rename(columns = {'STATE' : 'state'}, inplace = True)

In [15]:
df_cust.rename(columns = {'Age' : 'age'}, inplace = True)

In [16]:
df_cust.rename(columns = {'Gender' : 'sex'}, inplace = True)

In [17]:
df_cust.rename(columns = {'fam_status' : 'marital_status'}, inplace = True)

In [18]:
df_cust.rename(columns = {'n_dependants' : 'number_of_dependants'}, inplace = True)

In [19]:
# Check renamed columns

df_cust.columns

Index(['user_id', 'first_name', 'last_name', 'sex', 'state', 'age',
       'date_joined', 'number_of_dependants', 'marital_status', 'income'],
      dtype='object')

## Value counts and data types

In [20]:
# Descriptive stats on cust df

df_cust.describe()

,user_id,age,number_of_dependants,income
count,206209.000000,206209.000000,206209.000000,206209.000000
mean,103105.000000,49.501646,1.499823,94632.852548
std,59527.555167,18.480962,1.118433,42473.786988
min,1.000000,18.000000,0.000000,25903.000000
25%,51553.000000,33.000000,0.000000,59874.000000
50%,103105.000000,49.000000,1.000000,93547.000000
75%,154657.000000,66.000000,3.000000,124244.000000
max,206209.000000,81.000000,3.000000,593901.000000


In [21]:
df_cust['user_id'].value_counts(dropna = False)

26711     1
67322     1
173044    1
61044     1
98344     1
         ..
146847    1
154991    1
172193    1
184326    1
80148     1
Name: user_id, Length: 206209, dtype: int64

Count of user_ids matches, though will change user_id to str. Integer is faster, in theory would consult client before change.

In [22]:
# Change user_id to str in df_cust

df_cust['user_id'] = df_cust['user_id'].astype('str')

In [23]:
# Check for change

df_cust.dtypes

user_id                 object
first_name              object
last_name               object
sex                     object
state                   object
age                      int64
date_joined             object
number_of_dependants     int64
marital_status          object
income                   int64
dtype: object

In [24]:
# Check user_id data type in ords_prods df

ords_prods_merge_final['user_id'].dtypes

dtype('int64')

In [25]:
# Change user_id in ords_prods df to match for future merge

ords_prods_merge_final['user_id'] = ords_prods_merge_final['user_id'].astype('str')

In [26]:
# Check user_id data type in ords_prods df

ords_prods_merge_final['user_id'].dtypes

dtype('O')

In [27]:
df_cust['first_name'].value_counts(dropna = False)

NaN        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

Observation: 11259 NaN first names, will address below.
Data type needs to be changed from int64 to str

In [28]:
# Change first_name to str in df_cust

df_cust['first_name'] = df_cust['first_name'].astype('str')

In [29]:
# Check for change

df_cust['first_name'].dtypes

dtype('O')

In [30]:
df_cust['last_name'].value_counts(dropna = False)

Hamilton      252
Randall       248
Lamb          243
Pennington    243
Barnett       242
             ... 
Poole         172
Bauer         166
Pearson       164
Payne         163
Jordan        162
Name: last_name, Length: 1000, dtype: int64

In [31]:
df_cust['sex'].value_counts(dropna = False)

Male      104067
Female    102142
Name: sex, dtype: int64

As mentioned, not complete data due to lack of nonbinary, gender-diverse catagories- which should exist due to sample size >100

In [32]:
df_cust['state'].value_counts(dropna = False)

Florida                 4044
Colorado                4044
Illinois                4044
Alabama                 4044
District of Columbia    4044
Hawaii                  4044
Arizona                 4044
Connecticut             4044
California              4044
Indiana                 4044
Arkansas                4044
Alaska                  4044
Delaware                4044
Iowa                    4044
Idaho                   4044
Georgia                 4044
Wyoming                 4043
Mississippi             4043
Oklahoma                4043
Utah                    4043
New Hampshire           4043
Kentucky                4043
Maryland                4043
Rhode Island            4043
Massachusetts           4043
Michigan                4043
New Jersey              4043
Kansas                  4043
South Dakota            4043
Minnesota               4043
Tennessee               4043
New York                4043
Washington              4043
Louisiana               4043
Montana       

In [33]:
df_cust['age'].value_counts(dropna = False)

19    3329
55    3317
51    3317
56    3306
32    3305
      ... 
65    3145
25    3127
66    3114
50    3102
36    3101
Name: age, Length: 64, dtype: int64

In [34]:
df_cust['date_joined'].value_counts(dropna = False)

9/17/2018     213
2/10/2018     212
4/1/2019      211
9/21/2019     211
12/19/2017    210
             ... 
9/1/2018      141
1/22/2018     140
11/24/2017    139
7/18/2019     138
8/6/2018      128
Name: date_joined, Length: 1187, dtype: int64

In [35]:
df_cust['number_of_dependants'].value_counts(dropna = False)

0    51602
3    51594
1    51531
2    51482
Name: number_of_dependants, dtype: int64

In [36]:
df_cust['marital_status'].value_counts(dropna = False)

married                             144906
single                               33962
divorced/widowed                     17640
living with parents and siblings      9701
Name: marital_status, dtype: int64

In [37]:
df_cust['income'].value_counts(dropna = False)

57192     10
95891     10
95710     10
97532      9
98675      9
          ..
73141      1
71524      1
74408      1
44780      1
148828     1
Name: income, Length: 108012, dtype: int64

Observations: All data types allign other than user_id and first_name, which were changed to str

In [38]:
# Check for mixed data types 


for col in df_cust.columns.tolist():
    weird = (df_cust[[col]].applymap(type) != df_cust[[col]].iloc[0].apply(type)).any(axis= 1)
    if len (df_cust[weird]) > 0:
        print(col)

Note: prior to change for first_name to str, would be listed as a mixed type. Already addressed this issue above.

## Address NaN and missing values

In [39]:
# Locate missing values

df_cust.isnull().sum()

user_id                 0
first_name              0
last_name               0
sex                     0
state                   0
age                     0
date_joined             0
number_of_dependants    0
marital_status          0
income                  0
dtype: int64

In [40]:
# Identify NaN values within first_name

df_cust['first_name'].value_counts(dropna = False)

nan        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

In [41]:
df_cust_first_name_nan = df_cust[df_cust['first_name']=='nan']

In [42]:
df_cust_first_name_nan.shape

(11259, 10)

In [43]:
# Export subset of user_ids with first_name missing values

df_cust_first_name_nan.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_cust_first_name_nan.pkl'))

In [44]:
df_cust.shape

(206209, 10)

In [45]:
df_cust_first_name_nan.head()

,user_id,first_name,last_name,sex,state,age,date_joined,number_of_dependants,marital_status,income
53,76659,nan,Gilbert,Male,Colorado,26,1/1/2017,2,married,41709
73,13738,nan,Frost,Female,Louisiana,39,1/1/2017,0,single,82518
82,89996,nan,Dawson,Female,Oregon,52,1/1/2017,3,married,117099
99,96166,nan,Oconnor,Male,Oklahoma,51,1/1/2017,1,married,155673
105,29778,nan,Dawson,Female,Utah,63,1/1/2017,3,married,151819


In [46]:
# Replace nan with 'Unknown' for first_name given the rest of the customer information can be used to find patterns and the name of the customer is less relevant than all of their other data

df_cust['first_name'].fillna('Unknown', inplace =True)

# Replacement value should add to clarity in future analysis steps

In [47]:
# Check for value change

df_cust['first_name'].value_counts(dropna = False)

nan        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

Did not change values- will ask tutor for why

In [48]:
# checking for missing values a different way

df_cust.isnull().sum()

user_id                 0
first_name              0
last_name               0
sex                     0
state                   0
age                     0
date_joined             0
number_of_dependants    0
marital_status          0
income                  0
dtype: int64

Unsure of how to replace nan first_name values with 'Uknown', reached out to tutor for help

In [49]:
df_cust['first_name'].isnull().sum()

0

# Check for duplicates

In [50]:
# Check cust dataframe for duplicates
df_cust_dups = df_cust[df_cust.duplicated()]

In [51]:
df_cust_dups.shape

(0, 10)

No full duplicates found

In [52]:
# Recheck ords_prods_merge_final for duplicates

df_ords_prods_merge_final_dups = ords_prods_merge_final[ords_prods_merge_final.duplicated()]

In [53]:
df_ords_prods_merge_final_dups.shape

(0, 24)

# Final Consistency checks

In [54]:
df_cust.shape

(206209, 10)

In [55]:
df_cust.head()

,user_id,first_name,last_name,sex,state,age,date_joined,number_of_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [56]:
df_cust.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 206209 entries, 0 to 206208
Data columns (total 10 columns):
 #   Column                Non-Null Count   Dtype 
---  ------                --------------   ----- 
 0   user_id               206209 non-null  object
 1   first_name            206209 non-null  object
 2   last_name             206209 non-null  object
 3   sex                   206209 non-null  object
 4   state                 206209 non-null  object
 5   age                   206209 non-null  int64 
 6   date_joined           206209 non-null  object
 7   number_of_dependants  206209 non-null  int64 
 8   marital_status        206209 non-null  object
 9   income                206209 non-null  int64 
dtypes: int64(3), object(7)
memory usage: 15.7+ MB


In [57]:
ords_prods_merge_final.shape

(32404859, 24)

In [58]:
ords_prods_merge_final.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spending_flag,median_last_order,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [59]:
ords_prods_merge_final.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32404859 entries, 0 to 32404858
Data columns (total 24 columns):
 #   Column                 Dtype   
---  ------                 -----   
 0   order_id               int64   
 1   user_id                object  
 2   order_number           int64   
 3   order_day_of_week      int64   
 4   order_hour_of_day      int64   
 5   days_since_last_order  float64 
 6   product_id             int64   
 7   add_to_cart_order      int64   
 8   reordered              int64   
 9   product_name           object  
 10  aisle_id               int64   
 11  department_id          int64   
 12  prices                 float64 
 13  _merge                 category
 14  price_label            object  
 15  busiest_day            object  
 16  busiest_days           object  
 17  busiest_period_of_day  object  
 18  max_order              int64   
 19  loyalty_flag           object  
 20  avg_spent              float64 
 21  spending_flag          object

In [60]:
# Drop _merge flag from ords_prods_merge_final

ords_prods_merge_final = ords_prods_merge_final.drop(columns = ['_merge'])

In [61]:
ords_prods_merge_final.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,price_label,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag,avg_spent,spending_flag,median_last_order,frequency_flag
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Mid-range product,Regularly busy,Regularly busy days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Mid-range product,Regularly busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Average orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Mid-range product,Least busy,Slowest days,Most orders,10,New customer,6.367797,Low spender,20.5,Non-frequent customer


In [62]:
ords_prods_merge_final.shape

(32404859, 23)

# Merge dataframes

In [63]:
# Merge df_cust and ords_prods_merge_final data frames 

orders_products_all = ords_prods_merge_final.merge(df_cust, on = 'user_id', indicator = True)

In [64]:
# Check for shape of merge, should match ords_prods_merge_final plus 9 additional rows

orders_products_all.shape

(32404859, 33)

In [65]:
# Check value counts for full outer join

orders_products_all['_merge'].value_counts()

both          32404859
left_only            0
right_only           0
Name: _merge, dtype: int64

In [66]:
# Check columns

orders_products_all.head(50)

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,first_name,last_name,sex,state,age,date_joined,number_of_dependants,marital_status,income,_merge
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
5,3367565,1,6,2,7,19.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
6,550135,1,7,1,9,20.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
7,3108588,1,8,1,14,14.0,196,2,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
8,2295261,1,9,1,16,0.0,196,4,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both
9,2550362,1,10,4,8,30.0,196,1,1,Soda,...,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423,both


In [67]:
# Drop _merge column

orders_products_all = orders_products_all.drop(columns = ['_merge'])

In [68]:
orders_products_all.columns

Index(['order_id', 'user_id', 'order_number', 'order_day_of_week',
       'order_hour_of_day', 'days_since_last_order', 'product_id',
       'add_to_cart_order', 'reordered', 'product_name', 'aisle_id',
       'department_id', 'prices', 'price_label', 'busiest_day', 'busiest_days',
       'busiest_period_of_day', 'max_order', 'loyalty_flag', 'avg_spent',
       'spending_flag', 'median_last_order', 'frequency_flag', 'first_name',
       'last_name', 'sex', 'state', 'age', 'date_joined',
       'number_of_dependants', 'marital_status', 'income'],
      dtype='object')

# Export data sets

In [69]:
# Export cleaned customer data

df_cust.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_cust_clean.pkl'))

In [70]:
# Export updated ords_prods_merge_final data

ords_prods_merge_final.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge_final_update.pkl'))

Note: already exported file containing all Nan values for customer first_name

In [71]:
# Export fully merged data frame

orders_products_all.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all.pkl'))

# Revise customer data frame

Previous issue converting nan values to "Unknown", likely due to str data type. All other steps remain accurate. Will make correction below and export updated data frame.

In [74]:
# Import cust dataframe to test issue

df_cust_clean_2 = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_cust_clean.pkl'))

In [75]:
df_cust_clean_2.head()

,user_id,first_name,last_name,sex,state,age,date_joined,number_of_dependants,marital_status,income
0,26711,Deborah,Esquivel,Female,Missouri,48,1/1/2017,3,married,165665
1,33890,Patricia,Hart,Female,New Mexico,36,1/1/2017,0,single,59285
2,65803,Kenneth,Farley,Male,Idaho,35,1/1/2017,2,married,99568
3,125935,Michelle,Hicks,Female,Iowa,40,1/1/2017,0,single,42049
4,130797,Ann,Gilmore,Female,Maryland,26,1/1/2017,1,married,40374


In [76]:
df_cust_clean_2.shape

(206209, 10)

In [77]:
df_cust_clean_2['first_name'].value_counts(dropna = False)

nan        11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

In [78]:
# Convert all nan values to 'Unknown', maintaining string data type

df_cust_clean_2['first_name'] = df_cust_clean_2['first_name'].str.replace('nan', 'Unknown')

In [79]:
# Check accuracy

df_cust_clean_2['first_name'].value_counts(dropna = False)

Unknown    11259
Marilyn     2213
Barbara     2154
Todd        2113
Jeremy      2104
           ...  
Merry        197
Eugene       197
Garry        191
Ned          186
David        186
Name: first_name, Length: 208, dtype: int64

In [80]:
# Exporting file, though will need to make updates on merged data frame since that step already complete


df_cust_clean_2.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'df_cust_clean_2.pkl'))

# Revise orders_products_all dataframe (full) with updated nan values

In [81]:
# Check data frame

orders_products_all.head()

,order_id,user_id,order_number,order_day_of_week,order_hour_of_day,days_since_last_order,product_id,add_to_cart_order,reordered,product_name,...,frequency_flag,first_name,last_name,sex,state,age,date_joined,number_of_dependants,marital_status,income
0,2539329,1,1,2,8,NaN,196,1,0,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
1,2398795,1,2,3,7,15.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
2,473747,1,3,3,12,21.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
3,2254736,1,4,4,7,29.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423
4,431534,1,5,4,15,28.0,196,1,1,Soda,...,Non-frequent customer,Linda,Nguyen,Female,Alabama,31,2/17/2019,3,married,40423


In [82]:
# Check column with nan values to change

orders_products_all['first_name'].value_counts (dropna = False)

nan        1775118
Marilyn     355715
Todd        332574
Jeremy      327718
Barbara     325726
            ...   
Ned          29252
Carol        29103
Garry        26677
David        26055
Tammy        25441
Name: first_name, Length: 208, dtype: int64

In [83]:
# Replace nan values of 'first_name' with 'Unknown'

orders_products_all['first_name'] = orders_products_all['first_name'].str.replace('nan', 'Unknown')

In [84]:
# Check for accurate update

orders_products_all['first_name'].value_counts (dropna = False)

Unknown    1775118
Marilyn     355715
Todd        332574
Jeremy      327718
Barbara     325726
            ...   
Ned          29252
Carol        29103
Garry        26677
David        26055
Tammy        25441
Name: first_name, Length: 208, dtype: int64

In [85]:
# Export final data frame

orders_products_all.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_all_final.pkl'))

All values are now accurate and remain str data type. The small sample used for visualizations (small_sample_orders_products_all_2), customer dataframe (customer_clean_2), and now final data frame containing all relevant information (orders_products_all_final) are now updated. In the future knowing the str.replace function will eliminate the need for edited steps and redoing data frames.